##### CAMADA BRONZE - INGESTÃO DE DADOS #####

In [1]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import duckdb
import pandas as pd
import os
from datetime import datetime

In [3]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [9]:
arquivo = 'Z0019_02.csv' #'Z0019_01.csv' e 'Z0019_02.csv'
data_ingestao = datetime.now()
df = pd.read_csv(f'../landing/{arquivo}', sep=';')
df['arquivo'] = arquivo
df['data_ingestao'] = data_ingestao
df.head()

,NATBR,MAKTX,WERKS,MAINS,LABST,arquivo,data_ingestao
0,10004,SERRA,BT50,100,200,Z0019_02.csv,2025-11-29 23:14:45.331727
1,10005,MACHADO,BT10,100,60,Z0019_02.csv,2025-11-29 23:14:45.331727
2,10003,PREGO,BT10,100,60,Z0019_02.csv,2025-11-29 23:14:45.331727


In [10]:
print(datetime.now())

2025-11-29 23:14:50.703039


In [11]:
con.execute("""
CREATE TABLE IF NOT EXISTS bronze_produtos (
    NATBR VARCHAR,
    MAKTX VARCHAR,
    WERKS VARCHAR,
    MAINS VARCHAR,
    LABST VARCHAR,
    arquivo VARCHAR,
    data_ingestao TIMESTAMP)
""")
#validando a criação da tabela
con.execute("DESCRIBE bronze_produtos").df()

,column_name,column_type,null,key,default,extra
0,NATBR,VARCHAR,YES,None,None,None
1,MAKTX,VARCHAR,YES,None,None,None
2,WERKS,VARCHAR,YES,None,None,None
3,MAINS,VARCHAR,YES,None,None,None
4,LABST,VARCHAR,YES,None,None,None
5,arquivo,VARCHAR,YES,None,None,None
6,data_ingestao,TIMESTAMP,YES,None,None,None


In [12]:
con.execute("insert into bronze_produtos select * from df")

In [13]:
resultado = con.execute("select * from bronze_produtos").fetchdf()
display(resultado)

,NATBR,MAKTX,WERKS,MAINS,LABST,arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,Z0019_01.csv,2025-11-29 23:11:50.350603
1,10002,MARTELOS,BT50,100,1500,Z0019_01.csv,2025-11-29 23:11:50.350603
2,10003,PREGO,BT10,100,50,Z0019_01.csv,2025-11-29 23:11:50.350603
3,10004,SERRA,BT50,100,200,Z0019_02.csv,2025-11-29 23:14:45.331727
4,10005,MACHADO,BT10,100,60,Z0019_02.csv,2025-11-29 23:14:45.331727
5,10003,PREGO,BT10,100,60,Z0019_02.csv,2025-11-29 23:14:45.331727


In [14]:
con.execute("ALTER TABLE bronze_produtos RENAME TO bronze_Z0019;")

In [15]:
resultado = con.execute("select * from bronze_Z0019").fetchdf()
display(resultado)

,NATBR,MAKTX,WERKS,MAINS,LABST,arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,Z0019_01.csv,2025-11-29 23:11:50.350603
1,10002,MARTELOS,BT50,100,1500,Z0019_01.csv,2025-11-29 23:11:50.350603
2,10003,PREGO,BT10,100,50,Z0019_01.csv,2025-11-29 23:11:50.350603
3,10004,SERRA,BT50,100,200,Z0019_02.csv,2025-11-29 23:14:45.331727
4,10005,MACHADO,BT10,100,60,Z0019_02.csv,2025-11-29 23:14:45.331727
5,10003,PREGO,BT10,100,60,Z0019_02.csv,2025-11-29 23:14:45.331727


In [16]:
con.close()